In [1]:
import numpy as np
import pandas as pd
import pickle
from scipy import sparse

In [23]:
use_sel_freq_dic = pickle.load(open('user_seller_freq_new_id.pkl', 'rb')) 
use_mer_freq_dic = pickle.load(open('user_merchant_freq_new_id.pkl', 'rb'))

In [4]:
user_167079 = pd.read_csv('user_167079.csv')

In [26]:
user_num = 167079
sel_num = 10000
mer_num = 6802

In [7]:
user_in_mer_and_sel = user_167079['new_use_id'].tolist()

In [9]:
user_intersection_old_new_id = dict(zip(user_in_mer_and_sel, range(user_num)))
user_intersection_new_old_id = dict(zip(range(user_num), user_in_mer_and_sel))

In [31]:
new_old_id_maps = pickle.load(open('new_old_id_maps.pkl', 'rb'))

In [32]:
new_old_id_maps['user_intersection_old_new_id'] = user_intersection_old_new_id
new_old_id_maps['user_intersection_new_old_id'] = user_intersection_new_old_id

In [33]:
pickle.dump(new_old_id_maps, open('new_old_id_maps.pkl', 'wb'))

In [27]:
use_sel_freq = np.zeros((user_num, sel_num))
use_mer_freq = np.zeros((user_num, mer_num))

In [28]:
for i in range(user_num):
    user_id = user_intersection_new_old_id[i]
    for j, val in use_sel_freq_dic[user_id]:
        use_sel_freq[i, int(j)] = val

In [29]:
for i in range(user_num):
    user_id = user_intersection_new_old_id[i]
    for j, val in use_mer_freq_dic[user_id]:
        use_mer_freq[i, int(j)] = val

In [46]:
sparse_use_sel_freq = sparse.csc_matrix(use_sel_freq)

In [47]:
sparse.save_npz('/dev/use_sel_freq.npz', sparse_use_sel_freq) 

PermissionError: [Errno 13] Permission denied: '/dev/use_sel_freq.npz'

In [36]:
use_mer_freq[:10,:]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [2]:
user_num = 300
sel_num = 200
mer_num = 100
user_cls_num = 30
sel_cls_num = 20
mer_cls_num = 10

In [3]:
def normalize(a):
    return a / np.sum(a, axis = 1, keepdims = True)

In [4]:
user_cls = normalize(np.random.rand(user_num, user_cls_num)) #P(u|C_u)
sel_cls = normalize(np.random.rand(sel_num, sel_cls_num)) #P(s|C_s)
mer_cls = normalize(np.random.rand(mer_num, mer_cls_num)) #P(m|C_m)
freq_user_sel = np.random.rand(user_num, sel_num) # R(u, m)
freq_user_mer = np.random.rand(user_num, mer_num) # R(u, s)

In [5]:
user_sel_corr = np.dot((np.dot(freq_user_sel.T, user_cls) / np.sum(user_cls, axis = 0)).T, sel_cls) \
/ np.sum(sel_cls, axis = 0).reshape((1, sel_cls_num))
user_mer_corr = np.dot((np.dot(freq_user_mer.T, user_cls) / np.sum(user_cls, axis = 0)).T, mer_cls) \
/ np.sum(mer_cls, axis = 0).reshape((1, mer_cls_num))

In [6]:
sel_mer_corr = np.dot(user_sel_corr.T, user_mer_corr)

In [7]:
print(user_sel_corr.shape)
print(user_mer_corr.shape)
print(sel_mer_corr.shape)

(30, 20)
(30, 10)
(20, 10)


In [8]:
sel_mer_corr2 = np.dot(np.dot(sel_cls, sel_mer_corr), mer_cls.T)

In [9]:
print(sel_mer_corr2.shape)

(200, 100)


In [24]:
#### 验证vectorization是否正确
user_sel_corr2 = np.zeros((user_cls_num, sel_cls_num))
for i in range(sel_cls_num):
    deno = np.sum(sel_cls[:,i])
    for j in range(user_cls_num):
        tmp = 0
        for k in range(sel_num):
            no1 = 0
            deno1 = 0
            for l in range(user_num):
                no1 += user_cls[l,j] * freq_user_sel[l,k]
                deno1 += user_cls[l,j]
            tmp += no1/deno1*sel_cls[k,i]
        user_sel_corr2[j,i] = tmp/deno
np.allclose(user_sel_corr, user_sel_corr2)